In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld

URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_event_all = ld.getCOOs()
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13639 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13641 ( 0.1%) Users that have less than 1 test interactions


In [4]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Integer(200, 500),
    "shrink": Integer(0, 100),
    "similarity": Categorical(["cosine"]),
    "normalize": Categorical([True]),
}

In [5]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = ItemKNNCFRecommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation,
                                         evaluator_test=evaluator_test)

In [6]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [7]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [8]:
import os

output_folder_path = "result_experiments/1_Collaborative"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 50  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [9]:
!pip install tables

     |████████████████████████████████| 14.3 MB 5.1 MB/s            
     |████████████████████████████████| 471 kB 4.0 MB/s            


In [10]:
hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 269, 'shrink': 42, 'similarity': 'cosine', 'normalize': True}
ItemKNNCFRecommender: URM Detected 2 ( 0.0%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 18059 (100.0%), 480.55 column/sec. Elapsed time 37.58 sec
EvaluatorHoldout: Processed 13639 (100.0%) in 19.26 sec. Users per second: 708
SearchBayesianSkopt: New best config found. Config 0: {'topK': 269, 'shrink': 42, 'similarity': 'cosine', 'normalize': True} - results: PRECISION: 0.2179925, PRECISION_RECALL_MIN_DEN: 0.2194541, RECALL: 0.0475659, MAP: 0.1063518, MAP_MIN_DEN: 0.1069668, MRR: 0.4457299, NDCG: 0.2272166, F1: 0.0780921, HIT_RATE: 0.8821028, ARHR_ALL_HITS: 0.6854115, NOVELTY: 0.0052659, AVERAGE_POPULARITY: 0.6951557, DIVERSITY_MEAN_INTER_LIST: 0.7516636, DIVERSITY_HERFINDAHL: 0.9751608, COVERAGE_ITEM: 0.0458497, COVERAGE_ITEM_CORRECT: 0.0204884, COVERAGE_

EvaluatorHoldout: Processed 13641 (100.0%) in 23.89 sec. Users per second: 571
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'topK': 473, 'shrink': 34, 'similarity': 'cosine', 'normalize': True} - results:
CUTOFF: 10 - PRECISION: 0.2698043, PRECISION_RECALL_MIN_DEN: 0.2709660, RECALL: 0.0475027, MAP: 0.1424175, MAP_MIN_DEN: 0.1429602, MRR: 0.5152909, NDCG: 0.2813740, F1: 0.0807826, HIT_RATE: 0.9265450, ARHR_ALL_HITS: 0.8512568, NOVELTY: 0.0052467, AVERAGE_POPULARITY: 0.7087232, DIVERSITY_MEAN_INTER_LIST: 0.7219347, DIVERSITY_HERFINDAHL: 0.9721882, COVERAGE_ITEM: 0.0406999, COVERAGE_ITEM_CORRECT: 0.0196578, COVERAGE_USER: 0.9993407, COVERAGE_USER_CORRECT: 0.9259341, DIVERSITY_GINI: 0.0025875, SHANNON_ENTROPY: 5.7709545, RATIO_DIVERSITY_HERFINDAHL: 0.9725633, RATIO_DIVERSITY_GINI: 0.0104358, RATIO_SHANNON_ENTROPY: 0.4657669, RATIO_AVERAGE_POPULARITY: 3.4664704, RATIO_NOVELTY: 0.0308573, 


Iteration No: 5 ended. Evaluation done at random point.
Time taken: 55.7928
F

Similarity column 18059 (100.0%), 2443.45 column/sec. Elapsed time 7.39 sec
EvaluatorHoldout: Processed 13639 (100.0%) in 25.71 sec. Users per second: 530
SearchBayesianSkopt: Config 10 is suboptimal. Config: {'topK': 391, 'shrink': 82, 'similarity': 'cosine', 'normalize': True} - results: PRECISION: 0.2173473, PRECISION_RECALL_MIN_DEN: 0.2190112, RECALL: 0.0475567, MAP: 0.1066047, MAP_MIN_DEN: 0.1073425, MRR: 0.4466745, NDCG: 0.2273296, F1: 0.0780382, HIT_RATE: 0.8822494, ARHR_ALL_HITS: 0.6870179, NOVELTY: 0.0052409, AVERAGE_POPULARITY: 0.7089359, DIVERSITY_MEAN_INTER_LIST: 0.7272511, DIVERSITY_HERFINDAHL: 0.9727198, COVERAGE_ITEM: 0.0390387, COVERAGE_ITEM_CORRECT: 0.0176089, COVERAGE_USER: 0.9991941, COVERAGE_USER_CORRECT: 0.8815385, DIVERSITY_GINI: 0.0025580, SHANNON_ENTROPY: 5.7802135, RATIO_DIVERSITY_HERFINDAHL: 0.9730952, RATIO_DIVERSITY_GINI: 0.0103168, RATIO_SHANNON_ENTROPY: 0.4665142, RATIO_AVERAGE_POPULARITY: 3.4675107, RATIO_NOVELTY: 0.0308230, 

Iteration No: 11 ended. Eval

EvaluatorHoldout: Processed 13639 (100.0%) in 17.84 sec. Users per second: 765
SearchBayesianSkopt: New best config found. Config 16: {'topK': 382, 'shrink': 3, 'similarity': 'cosine', 'normalize': True} - results: PRECISION: 0.2183958, PRECISION_RECALL_MIN_DEN: 0.2200181, RECALL: 0.0478843, MAP: 0.1069369, MAP_MIN_DEN: 0.1076239, MRR: 0.4468433, NDCG: 0.2280455, F1: 0.0785468, HIT_RATE: 0.8862087, ARHR_ALL_HITS: 0.6883448, NOVELTY: 0.0052676, AVERAGE_POPULARITY: 0.6976409, DIVERSITY_MEAN_INTER_LIST: 0.7432808, DIVERSITY_HERFINDAHL: 0.9743226, COVERAGE_ITEM: 0.0478986, COVERAGE_ITEM_CORRECT: 0.0212083, COVERAGE_USER: 0.9991941, COVERAGE_USER_CORRECT: 0.8854945, DIVERSITY_GINI: 0.0029631, SHANNON_ENTROPY: 5.9308927, RATIO_DIVERSITY_HERFINDAHL: 0.9746986, RATIO_DIVERSITY_GINI: 0.0119506, RATIO_SHANNON_ENTROPY: 0.4786753, RATIO_AVERAGE_POPULARITY: 3.4122650, RATIO_NOVELTY: 0.0309800, 

EvaluatorHoldout: Processed 13641 (100.0%) in 18.28 sec. Users per second: 746
SearchBayesianSkopt: Conf

Similarity column 18059 (100.0%), 2424.16 column/sec. Elapsed time 7.45 sec
EvaluatorHoldout: Processed 13639 (100.0%) in 17.08 sec. Users per second: 799
SearchBayesianSkopt: Config 21 is suboptimal. Config: {'topK': 288, 'shrink': 1, 'similarity': 'cosine', 'normalize': True} - results: PRECISION: 0.2189090, PRECISION_RECALL_MIN_DEN: 0.2204161, RECALL: 0.0478298, MAP: 0.1068524, MAP_MIN_DEN: 0.1075664, MRR: 0.4470248, NDCG: 0.2281968, F1: 0.0785066, HIT_RATE: 0.8848889, ARHR_ALL_HITS: 0.6878254, NOVELTY: 0.0052796, AVERAGE_POPULARITY: 0.6905520, DIVERSITY_MEAN_INTER_LIST: 0.7561653, DIVERSITY_HERFINDAHL: 0.9756110, COVERAGE_ITEM: 0.0505565, COVERAGE_ITEM_CORRECT: 0.0229249, COVERAGE_USER: 0.9991941, COVERAGE_USER_CORRECT: 0.8841758, DIVERSITY_GINI: 0.0032451, SHANNON_ENTROPY: 6.0491574, RATIO_DIVERSITY_HERFINDAHL: 0.9759875, RATIO_DIVERSITY_GINI: 0.0130882, RATIO_SHANNON_ENTROPY: 0.4882204, RATIO_AVERAGE_POPULARITY: 3.3775923, RATIO_NOVELTY: 0.0310502, 

Iteration No: 22 ended. Searc

EvaluatorHoldout: Processed 13639 (100.0%) in 19.93 sec. Users per second: 684
SearchBayesianSkopt: Config 27 is suboptimal. Config: {'topK': 395, 'shrink': 12, 'similarity': 'cosine', 'normalize': True} - results: PRECISION: 0.2184984, PRECISION_RECALL_MIN_DEN: 0.2201879, RECALL: 0.0477791, MAP: 0.1070771, MAP_MIN_DEN: 0.1078255, MRR: 0.4470269, NDCG: 0.2281721, F1: 0.0784119, HIT_RATE: 0.8806364, ARHR_ALL_HITS: 0.6886239, NOVELTY: 0.0052625, AVERAGE_POPULARITY: 0.6997611, DIVERSITY_MEAN_INTER_LIST: 0.7400946, DIVERSITY_HERFINDAHL: 0.9740040, COVERAGE_ITEM: 0.0447976, COVERAGE_ITEM_CORRECT: 0.0204884, COVERAGE_USER: 0.9991941, COVERAGE_USER_CORRECT: 0.8799267, DIVERSITY_GINI: 0.0028655, SHANNON_ENTROPY: 5.9008760, RATIO_DIVERSITY_HERFINDAHL: 0.9743799, RATIO_DIVERSITY_GINI: 0.0115573, RATIO_SHANNON_ENTROPY: 0.4762527, RATIO_AVERAGE_POPULARITY: 3.4226356, RATIO_NOVELTY: 0.0309502, 

Iteration No: 28 ended. Search finished for the next optimal point.
Time taken: 28.1367
Function value o

EvaluatorHoldout: Processed 13641 (100.0%) in 18.14 sec. Users per second: 752
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'topK': 367, 'shrink': 0, 'similarity': 'cosine', 'normalize': True} - results:
CUTOFF: 10 - PRECISION: 0.2717323, PRECISION_RECALL_MIN_DEN: 0.2729635, RECALL: 0.0480414, MAP: 0.1436569, MAP_MIN_DEN: 0.1441524, MRR: 0.5156310, NDCG: 0.2829142, F1: 0.0816478, HIT_RATE: 0.9281578, ARHR_ALL_HITS: 0.8548209, NOVELTY: 0.0052717, AVERAGE_POPULARITY: 0.6955860, DIVERSITY_MEAN_INTER_LIST: 0.7466073, DIVERSITY_HERFINDAHL: 0.9746553, COVERAGE_ITEM: 0.0495598, COVERAGE_ITEM_CORRECT: 0.0226480, COVERAGE_USER: 0.9993407, COVERAGE_USER_CORRECT: 0.9275458, DIVERSITY_GINI: 0.0030355, SHANNON_ENTROPY: 5.9571182, RATIO_DIVERSITY_HERFINDAHL: 0.9750314, RATIO_DIVERSITY_GINI: 0.0122429, RATIO_SHANNON_ENTROPY: 0.4807920, RATIO_AVERAGE_POPULARITY: 3.4022144, RATIO_NOVELTY: 0.0310043, 


Iteration No: 33 ended. Search finished for the next optimal point.
Time taken

EvaluatorHoldout: Processed 13639 (100.0%) in 28.80 sec. Users per second: 474
SearchBayesianSkopt: Config 38 is suboptimal. Config: {'topK': 484, 'shrink': 99, 'similarity': 'cosine', 'normalize': True} - results: PRECISION: 0.2165335, PRECISION_RECALL_MIN_DEN: 0.2182751, RECALL: 0.0474766, MAP: 0.1058546, MAP_MIN_DEN: 0.1065981, MRR: 0.4455336, NDCG: 0.2264248, F1: 0.0778778, HIT_RATE: 0.8818828, ARHR_ALL_HITS: 0.6840920, NOVELTY: 0.0052282, AVERAGE_POPULARITY: 0.7171410, DIVERSITY_MEAN_INTER_LIST: 0.7086071, DIVERSITY_HERFINDAHL: 0.9708555, COVERAGE_ITEM: 0.0358824, COVERAGE_ITEM_CORRECT: 0.0166676, COVERAGE_USER: 0.9991941, COVERAGE_USER_CORRECT: 0.8811722, DIVERSITY_GINI: 0.0023262, SHANNON_ENTROPY: 5.6552405, RATIO_DIVERSITY_HERFINDAHL: 0.9712302, RATIO_DIVERSITY_GINI: 0.0093819, RATIO_SHANNON_ENTROPY: 0.4564278, RATIO_AVERAGE_POPULARITY: 3.5076432, RATIO_NOVELTY: 0.0307481, 

Iteration No: 39 ended. Search finished for the next optimal point.
Time taken: 37.0758
Function value o

SearchBayesianSkopt: Config 44 is suboptimal. Config: {'topK': 412, 'shrink': 0, 'similarity': 'cosine', 'normalize': True} - results: PRECISION: 0.2185497, PRECISION_RECALL_MIN_DEN: 0.2202821, RECALL: 0.0478849, MAP: 0.1068283, MAP_MIN_DEN: 0.1075546, MRR: 0.4453545, NDCG: 0.2279452, F1: 0.0785576, HIT_RATE: 0.8811496, ARHR_ALL_HITS: 0.6871266, NOVELTY: 0.0052672, AVERAGE_POPULARITY: 0.6987862, DIVERSITY_MEAN_INTER_LIST: 0.7402717, DIVERSITY_HERFINDAHL: 0.9740217, COVERAGE_ITEM: 0.0474002, COVERAGE_ITEM_CORRECT: 0.0209314, COVERAGE_USER: 0.9991941, COVERAGE_USER_CORRECT: 0.8804396, DIVERSITY_GINI: 0.0029280, SHANNON_ENTROPY: 5.9141238, RATIO_DIVERSITY_HERFINDAHL: 0.9743976, RATIO_DIVERSITY_GINI: 0.0118092, RATIO_SHANNON_ENTROPY: 0.4773220, RATIO_AVERAGE_POPULARITY: 3.4178668, RATIO_NOVELTY: 0.0309778, 

Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 27.8457
Function value obtained: -0.1068
Current minimum: -0.1072
Iteration No: 46 started. Searching fo

SearchBayesianSkopt: Best config evaluated with evaluator_test with constructor data for final test. Config: {'topK': 367, 'shrink': 0, 'similarity': 'cosine', 'normalize': True} - results:
CUTOFF: 10 - PRECISION: 0.3429587, PRECISION_RECALL_MIN_DEN: 0.3442850, RECALL: 0.0584674, MAP: 0.2076600, MAP_MIN_DEN: 0.2083214, MRR: 0.6135911, NDCG: 0.3627045, F1: 0.0999034, HIT_RATE: 0.9487574, ARHR_ALL_HITS: 1.1106903, NOVELTY: 0.0052881, AVERAGE_POPULARITY: 0.6747180, DIVERSITY_MEAN_INTER_LIST: 0.7838086, DIVERSITY_HERFINDAHL: 0.9783751, COVERAGE_ITEM: 0.0463481, COVERAGE_ITEM_CORRECT: 0.0244753, COVERAGE_USER: 0.9993407, COVERAGE_USER_CORRECT: 0.9481319, DIVERSITY_GINI: 0.0033097, SHANNON_ENTROPY: 6.1448267, RATIO_DIVERSITY_HERFINDAHL: 0.9787527, RATIO_DIVERSITY_GINI: 0.0133443, RATIO_SHANNON_ENTROPY: 0.4959327, RATIO_AVERAGE_POPULARITY: 3.3251606, RATIO_NOVELTY: 0.0248799, 


SearchBayesianSkopt: Saving model in result_experiments/1_CollaborativeItemKNNCFRecommender

ItemKNNCFRecommender: 

In [14]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

dict_keys(['exception_list', 'result_on_test_df', 'result_on_last', 'time_on_train_total', 'hyperparameters_df', 'time_df', 'algorithm_name_recommender', 'metric_to_optimize', 'algorithm_name_search', 'hyperparameters_best_index', 'cutoff_to_optimize', 'time_on_validation_total', 'result_on_validation_best', 'hyperparameters_best', 'time_on_last_df', 'time_on_validation_avg', 'time_on_test_avg', 'time_on_test_total', 'result_on_test_best', 'result_on_validation_df', 'time_on_train_avg'])

In [15]:
hyperparameters_df = search_metadata["hyperparameters_best"]
hyperparameters_df

{'topK': 367, 'shrink': 0, 'similarity': 'cosine', 'normalize': True}

In [13]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,
0,10,0.217993,0.219454,0.047566,0.106352,0.106967,0.44573,0.227217,0.078092,0.882103,0.685412,...,0.020488,0.999194,0.881392,0.003052,6.000148,0.975537,0.01231,0.484265,3.400109,0.03097
1,10,0.21874,0.220373,0.047965,0.10654,0.107313,0.445206,0.227851,0.078678,0.884156,0.68588,...,0.021264,0.999194,0.883443,0.002936,5.935133,0.974796,0.011842,0.479018,3.413934,0.030951
2,10,0.217912,0.219666,0.047643,0.106154,0.106909,0.44352,0.227035,0.07819,0.882396,0.683496,...,0.017941,0.999194,0.881685,0.002511,5.748376,0.972448,0.010127,0.463945,3.478116,0.030812
3,10,0.218168,0.21973,0.047889,0.106448,0.107218,0.44751,0.227717,0.078539,0.884376,0.686872,...,0.021485,0.999194,0.883663,0.003005,5.96391,0.975134,0.012118,0.48134,3.404027,0.030975
4,10,0.217787,0.21954,0.047812,0.10661,0.107328,0.446227,0.227635,0.07841,0.883716,0.687477,...,0.018827,0.999194,0.883004,0.002583,5.770068,0.972559,0.01042,0.465695,3.46674,0.030857
5,10,0.216959,0.218467,0.047362,0.105697,0.10639,0.444904,0.226394,0.077751,0.881296,0.683029,...,0.018163,0.999194,0.880586,0.002672,5.831424,0.973715,0.010776,0.470647,3.450174,0.030857
6,10,0.218249,0.219645,0.047656,0.105669,0.106297,0.44225,0.226585,0.07823,0.881883,0.681017,...,0.019436,0.999194,0.881172,0.002945,5.963267,0.975116,0.011879,0.481288,3.41751,0.030916
7,10,0.218007,0.2196,0.047732,0.106746,0.107486,0.448637,0.227939,0.078316,0.884156,0.68878,...,0.020378,0.999194,0.883443,0.002895,5.915351,0.974545,0.011677,0.477421,3.41853,0.03095
8,10,0.217567,0.219399,0.04763,0.106352,0.107131,0.445317,0.227214,0.078151,0.882103,0.685281,...,0.017332,0.999194,0.881392,0.002449,5.720071,0.972101,0.009879,0.46166,3.487233,0.03079
